In [ ]:
#@title import libraries
import numpy as np
import numpy.linalg as npla
import matplotlib.pyplot as plt

from numpy import linalg as LA

In [ ]:
#@title define functions

def f1(x):
    return np.exp(x-3) - x/2 - 2

def f2(x):
    return -0.5 * (1+ (1/(2**x)) ) * np.sin(2*x) 

def f3(x):
     return (1 - x[0])**2+ 100*((x[1]-x[0]**2)**2)

In [ ]:
#@title Gradient of functions

def f1_dx(x):
    return np.exp(x - 3) - 0.5

def f2_dx(x):
    df2_1 = 2**(-x) * np.log(2) * 0.5 * np.sin(2*x)
    df2_2 = np.cos(2*x)/(2**(x))  + np.cos(2*x)
    return df2_1 - df2_2

def f3_dx(x):
    df1 = -2*(1 - x[0]) - (400*x[0])*(x[1] - (x[0]**2))
    df2 = 200*(x[1] - (x[0]**2))
    return np.array([df1, df2])

In [ ]:
#@title title Hessian of functions

def f1_hessian(x):
    return np.exp(x-3)

def f2_hessian(x):
    hessian_1 = 2**(-x) * np.cos(2*x) * np.log(2) - 2**(-x-1) * (np.log(2)**2) * np.sin(2*x)
    hessian_2 = 2**(2*x +1 ) * np.sin(2*x) + 2**(x+1) * np.sin(2*x)
    hessian_3 = 2**(x) * np.log(2) * np.cos(2*x)
    return hessian_1 + (2**(-2*x) * (hessian_2 + hessian_3))

def f3_hessian(x):
    d2f_dx2 = 2 - 400*x[1] + 1200 * (x[0]**2)
    d2f_dyx = -400*x[0]
    d2f_dy2 = 200
    return(np.matrix([[d2f_dx2, d2f_dyx], [d2f_dyx, d2f_dy2]]))
    

In [ ]:
#@title LINE SEARCH STEP SIZE

def backtrack2(x0, f, fdx, t = 1, alpha=0.2, beta=0.8):
    while f(x0 - t*fdx(x0)) > f(x0) + alpha * t * np.dot(fdx(x0).T, -1*fdx(x0)):
        t *= beta
    return t

In [ ]:
#@title Steepest Descent

def grad(point, max_iter, f, fdx):
    iter = 1
    points = []
    values = []
    step_sizes = []
    while (np.linalg.norm(fdx(point)) > 0.000001):

        t = backtrack2(point, f, fdx)
        point = point - np.dot(t, fdx(point))
        print(point, f(point), fdx(point), iter)
        iter += 1
        if iter > max_iter:
            break
        points.append(point)
        values.append(f(point))
        step_sizes.append(t)

    return points, values, step_sizes


In [ ]:
#@title Newton's Method

def lambda_sq(fdx, Hessian, point):
    val_hessian = Hessian(point)
    if val_hessian.ndim < 2:
        lambda_sq = fdx(point) * val_hessian * fdx(point)
    else:
        lambda_sq = np.dot(np.dot(fdx(point), npla.pinv(val_hessian)), fdx(point).T)
    return (np.asscalar(lambda_sq))

def delta_x(fdx, Hessian, point):
    val_hessian = Hessian(point)
    if val_hessian.ndim < 2:
        delta_x = val_hessian * fdx(point)
    else:
        delta_x = np.dot(-npla.pinv(Hessian(point)), fdx(point).T)
    return (delta_x)

def newtons_method(x, f, fdx, Hessian, eps=0.00001, max_iters=20000, is_hessian=False):
    iters = 1
    points = []
    values = []
    step_sizes = []
    lmb_sq = lambda_sq(fdx, Hessian, x)
    # while((lmb_sq/2.0) > eps):
    while True:
        dlt_x = delta_x(fdx, Hessian, x)
        t = backtrack2(x, f, fdx)
        value = np.array((x + np.dot(t , dlt_x)))
        if is_hessian: 
            x = value[0]
        else:
            x = value
        lmb_sq = lambda_sq(fdx, Hessian, x)
        
        iters += 1
        
        if(iters > max_iters):
            break
        points.append(x)
        values.append(f(x))
        step_sizes.append(t)
    return points, values, step_sizes

In [ ]:
#@title simulation

point = [2.0, 2.0]
x = 4
# points, values, step_sizes = grad(point, max_iter=20000, f=f3, fdx=f3_dx)
points, values, step_sizes = newtons_method(x, f1, f1_dx, f1_hessian, eps=0.00001,
                                            max_iters=20000, is_hessian=False)


In [ ]:
#@title simple newton method

def f(x):
    return -0.5 * (1+ (1/(2**x)) ) * np.sin(2*x) 

def f_grad(x):
    df2_1 = 2**(-x) * np.log(2) * 0.5 * np.sin(2*x)
    df2_2 = np.cos(2*x)/(2**(x))  + np.cos(2*x)
    return df2_1 - df2_2

def f_grad_2(x):
    hessian_1 = 2**(-x) * np.cos(2*x) * np.log(2) - 2**(-x-1) * (np.log(2)**2) * np.sin(2*x)
    hessian_2 = 2**(2*x +1 ) * np.sin(2*x) + 2**(x+1) * np.sin(2*x)
    hessian_3 = 2**(x) * np.log(2) * np.cos(2*x)
    return hessian_1 + (2**(-2*x) * (hessian_2 + hessian_3))


def BacktrackingLineSearch(x0):
    alpha = 1
    x = x0
    rho = 0.8
    c = 1e-4

    while f( x + alpha * (-f_grad(x)) ) > f(x) + c * alpha * f_grad(x) * (-f_grad(x)) :
        alpha *= rho

    return alpha

def NewtonMethod():
    x0 = 0
    curve_y = [f(x0)]
    curve_x = [x0]

    step_sizes = []

    lambda_squre = 1
    error = 1e-4
    while lambda_squre/2 > error:
        stepSize = BacktrackingLineSearch(x0)

        xnt=- (1/f_grad_2(x0))*(f_grad(x0))
        x0 = x0 + stepSize * xnt

        y1 = f(x0)
        lambda_squre=(f_grad(x0)) * (1/(f_grad_2(x0))) * (f_grad(x0))

        curve_x.append(x0)
        curve_y.append(y1)
        step_sizes.append(stepSize)
    return curve_x, curve_y, step_sizes

if __name__ == "__main__":
    curve_x, curve_y, step_sizes = NewtonMethod()

In [ ]:
#@title Plot function values

# values = curve_y
plt.plot(values, '-r')
# plt.ylabel('Function Values')
plt.xlabel('Iterations')
plt.grid()
plt.title("Function Values")

In [ ]:
#@title plot distance to distance to optimal point

points = curve_x
points = points - points[-1]
# dist = LA.norm(points, axis=1)
plt.figure()
plt.plot(dist, '*b')
# plt.ylabel('distnace')
plt.xlabel('Iterations')
plt.grid()
plt.title("Distance to optimal point")
# plt.ylim(0, 2)

In [ ]:
#@title plot distance to step sizes

plt.figure()
plt.plot(step_sizes, 'og')
# plt.ylabel('distnace')
plt.xlabel('Iterations')
plt.grid()
plt.title("Step size")
# plt.ylim(0, 2)